<h2>Mount Drive</h2>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<h2>Navigate to mid slices and set up tuned models directory</h2>

In [0]:
import os
import pandas as pd
from time import sleep

os.chdir('drive/My Drive/mid_slices')
cwd = os.getcwd()

#create models directory if it doesn't already exist
model_dir = 'tuned_models'
models_path = os.path.join(cwd,model_dir)
try:
    os.mkdir(models_path)
except:
    print("WARNING: tuned models directory already exists")

#load transition metadata as datframe
df = pd.read_csv(os.path.join(cwd,'mid_slice_metadata.csv'))

#display unique labels
print(df['Label'].unique())
print(df['Soft Labels'].unique())

FileNotFoundError: ignored

<h2>Get Transition slice chunk paths</h2>

In [0]:
import os
import numpy as np

#method to extract slice number
def get_slices_num(slice):
  num = int(slice[5:-4])
  return(num)

os.chdir('drive/My Drive/mid_slices')
print(os.getcwd())

#get slice chunk filepaths
np_slices = {}
for (dir,root,files) in os.walk(os.getcwd()):
  for file in files:
    if '.npy' in file:
      num = get_slices_num(file)
      np_slices[num] = file

#sort slices
sorted_slices = sorted(np_slices.keys())





/content/drive/My Drive/mid_slices
{6: 'chunk6.npy', 7: 'chunk7.npy', 10: 'chunk10.npy', 8: 'chunk8.npy', 9: 'chunk9.npy', 5: 'chunk5.npy', 11: 'chunk11.npy', 12: 'chunk12.npy', 13: 'chunk13.npy', 14: 'chunk14.npy', 15: 'chunk15.npy', 16: 'chunk16.npy', 17: 'chunk17.npy', 18: 'chunk18.npy', 19: 'chunk19.npy', 20: 'chunk20.npy', 21: 'chunk21.npy', 22: 'chunk22.npy', 23: 'chunk23.npy', 24: 'chunk24.npy', 25: 'chunk25.npy', 26: 'chunk26.npy', 27: 'chunk27.npy', 28: 'chunk28.npy', 29: 'chunk29.npy', 30: 'chunk30.npy', 31: 'chunk31.npy', 32: 'chunk32.npy', 33: 'chunk33.npy', 34: 'chunk34.npy', 35: 'chunk35.npy', 36: 'chunk36.npy', 37: 'chunk37.npy', 38: 'chunk38.npy', 39: 'chunk39.npy', 40: 'chunk40.npy', 41: 'chunk41.npy', 42: 'chunk42.npy', 43: 'chunk43.npy', 44: 'chunk44.npy', 45: 'chunk45.npy', 46: 'chunk46.npy', 47: 'chunk47.npy', 48: 'chunk48.npy', 49: 'chunk49.npy', 50: 'chunk50.npy', 51: 'chunk51.npy', 52: 'chunk52.npy', 53: 'chunk53.npy', 54: 'chunk54.npy', 55: 'chunk55.npy', 56: '

<h2>Load CT slice image arrays</h2>



In [0]:
import time
import cv2
from numpy import save, asarray
from keras.preprocessing.image import array_to_img,save_img
import tensorflow as tf
from time import sleep

#method to reshape image dimensions
def decode_img(img,IMG_WIDTH,IMG_HEIGHT):
  return(cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT)))

#start counter
start = time.perf_counter()

#data array
data = []

#load every slice per chunk
for chunk_ in  sorted_slices:

  #load np slice chunk
  chunk = np.load(np_slices[chunk_])
  
  #format each slice to (224,224,3) if not already in the format
  for slice_ in chunk:
    slice_ = decode_img(scan,224,224)

    if slice_.ndim < 3:
      slice_ = np.asarray([scan]*3)
      slice_ = slice_.reshape(224,224,3)
    
    #append slice to data array
    data.append(slice_)
  
#finish counter
finish = time.perf_counter()
print(f'this process took {finish-start} seconds')



  





  






/content/drive/My Drive/mid_slices
chunk5.npy with shape (106, 512, 512, 3) loaded at: 1300.619780654
data length: 0


chunk6.npy with shape (106, 512, 512, 3) loaded at: 1301.061699985
data length: 106


chunk7.npy with shape (106, 512, 512, 3) loaded at: 1301.461012015
data length: 212


chunk8.npy with shape (106, 512, 512, 3) loaded at: 1301.873569543
data length: 318


chunk9.npy with shape (106, 512, 512, 3) loaded at: 1302.311760171
data length: 424


chunk10.npy with shape (106, 512, 512, 3) loaded at: 1302.714486868
data length: 530


chunk11.npy with shape (106, 512, 512, 3) loaded at: 1303.14820651
data length: 636


chunk12.npy with shape (106, 512, 512, 3) loaded at: 1303.554250729
data length: 742


chunk13.npy with shape (106, 512, 512, 3) loaded at: 1303.970615222
data length: 848


chunk14.npy with shape (106, 512, 512, 3) loaded at: 1304.379421783
data length: 954


chunk15.npy with shape (106, 512, 512, 3) loaded at: 1304.783659072
data length: 1060


chunk16.npy wit

<h2>Divide Transition dataset</h2>

In [0]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

#start counter
start = time.perf_counter()

#exclude corrupted 424 slices 
labels = df['Soft Labels']       
labels =labels[424:]

#split 80:20 train test data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20)

#finish counter
finish = time.perf_counter()
print(f'this process took {finish-start} seconds')


424      7
425      0
426      0
427      0
428      0
        ..
10595    6
10596    5
10597    5
10598    5
10599    5
Name: Soft Labels, Length: 10176, dtype: int64
this process took 0.09296261600002254 seconds
10176


<h2>Convert Data to Numpy Arrays</h2>

In [0]:
from keras.preprocessing.image import ImageDataGenerator

#convert divided data into numpy arrays
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)





<h2>Initialize Networks</h2>

In [0]:
from keras.applications import MobileNetV2, InceptionV3, ResNet50V2, VGG16, VGG19, Xception, ResNet101V2, DenseNet121, ResNet152V2, DenseNet169, DenseNet201 ##
from keras import datasets, layers, models
from keras.models import load_model
import matplotlib.pyplot as plt
import os
import numpy as npf
from time import sleep

#input dimensions
img_shape = (224,224,3)


#model initialization
densenet121 = DenseNet121(weights='imagenet', include_top=False, input_shape=img_shape)
densenet169 = DenseNet169(weights='imagenet', include_top=False, input_shape=img_shape)
resnet50 = ResNet50V2(weights='imagenet', include_top=False, input_shape=img_shape)
resnet101 = ResNet101V2(weights='imagenet', include_top=False, input_shape=img_shape)

#create model list
model_list = [densenet121,densenet169,resnet50,resnet101]


/content/drive/My Drive/mid_slices
171319296/171317808 [==============================] - 11s 0us/step


<h2>Freeze layers</h2>

In [0]:
from keras.layers.convolutional import Conv2D

#for each model freeze shallow half of conv layers
for model in model_list:

    #get model name
    model_config = model.get_config()
    name = model_config['name']
    print(name)

    #freeze shallower half of all convolutional layers
    conv_layers = []
    for layer in model.layers:
        if isinstance(layer,Conv2D):
          layer.trainable = True
          conv_layers.append(layer)

    n_freeze = len(conv_layers)//2
    for layer in conv_layers[:n_freeze]:
       layer.trainable = False 

    for layer in model.layers:
        # layer.trainable = True
        print(layer,layer.trainable,layer.output_shape)
        
    

    

Using TensorFlow backend.


NameError: ignored

<h2>Add randomly initialized FC layers to Source Model</h2>

In [0]:
import numpy as np
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
tuned_models = []
for model in model_list:
    
    #get model name
    model_config = model.get_config()
    name = model_config['name']
    
    #create transition model and add source model
    tuned_model = models.Sequential()
    tuned_model.add(model)


    #random initialization fixed for reproducibility
    np.random.seed(1000)
    

    #add output layers for soft labels
    if "resnet" in name:
    #---------------RESNET LAYERS---------------
      tuned_model.add(AveragePooling2D())
      tuned_model.add(layers.Flatten())
      tuned_model.add(layers.Dense(1000, activation='relu'))
      tuned_model.add(layers.Dense(16, activation='softmax'))

    elif "densenet" in name:
    #---------------DENSENET LAYERS---------------
      tuned_model.add(GlobalAveragePooling2D())
      tuned_model.add(layers.Dense(1000, activation='relu'))
      tuned_model.add(layers.Dense(16, activation='softmax'))

    ####uncomment to add output layers for hard labels####
    # if "resnet" in name:
    # #---------------RESNET LAYERS---------------
    #   tuned_model.add(AveragePooling2D())
    #   tuned_model.add(layers.Flatten())
    #   tuned_model.add(layers.Dense(1000, activation='relu'))
    #   tuned_model.add(layers.Dense(1, activation='sigmoid'))

    # elif "densenet" in name:
    # #---------------DENSENET LAYERS---------------
    #   tuned_model.add(GlobalAveragePooling2D())
    #   tuned_model.add(layers.Dense(1000, activation='relu'))
    #   tuned_model.add(layers.Dense(1, activation='sigmoid'))


    #add model to model list
    tuned_models.append(tuned_model)

#display each model summary
[print(model.summary()) for model in tuned_models]

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1025000   
_________________________________________________________________
dense_6 (Dense)              (None, 16)                16016     
Total params: 8,078,520
Trainable params: 5,626,168
Non-trainable params: 2,452,352
_________________________________________________________________
None
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 7, 7, 1664)        12642880  
_________

[None, None, None, None]

<h2>Learn the Transition task</h2>

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD, Adam, Adadelta
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.metrics import AUC
from keras.backend import manual_variable_initialization
 

#start counter
start = time.perf_counter()

#optimizer
if ("resnet" in model.layers[0].name or "densenet"in model.layers[0].name):
  opt = SGD() 
else:
  opt = Adadelta()


for model in tuned_models[1:]:

  #create save path
  name = model.layers[0].name +"_soft_labels_15epochs.hdf5"
  path = os.path.join(model_dir,name)
 
  #compile model
  model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])


  #train model and save model with highest validation accuracy
  callbacks = [ModelCheckpoint(filepath=path, verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')]
  model.fit(X_train,y_train,verbose=1,use_multiprocessing=True,epochs=15,validation_data=(X_test,y_test),callbacks=callbacks,shuffle=True)


finish = time.perf_counter()
print(f'this process took {finish-start} seconds')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 7, 7, 1664)        12642880  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1664)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1000)              1665000   
_________________________________________________________________
dense_8 (Dense)              (None, 16)                16016     
Total params: 14,323,896
Trainable params: 10,150,200
Non-trainable params: 4,173,696
_________________________________________________________________
None
compiled
Train on 8140 samples, validate on 2036 samples
Epoch 1/15
8140/8140 [==============================] - 208s 26ms/step - loss: 1.2056 - accuracy: 0.6243 - val_loss: 2.5714 - val_accuracy: 0.4136

Epoch 00001: val_accur